In [1]:
import tensorflow as tf
import os
import glob
import tarfile
import numpy as np
from scipy.io import loadmat
from shutil import copyfile, rmtree
import sys
import json
if sys.version_info[0] >= 3:
    from urllib.request import urlretrieve
else:
    from urllib import urlretrieve

In [2]:
def download_file(url, dest=None):
    if not dest:
        dest = os.path.join(data_path, url.split('/')[-1])
    urlretrieve(url, dest)

In [3]:
data_path = 'flower'

In [4]:
if not os.path.exists(data_path):
    os.mkdir(data_path)
flowers_archive_path = os.path.join(data_path, '102flowers.tgz')
if not os.path.isfile(flowers_archive_path):
    print ('Downloading images...')
    download_file('http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz')
    tarfile.open(flowers_archive_path).extractall(path=data_path)

image_labels_path = os.path.join(data_path, 'imagelabels.mat')
if not os.path.isfile(image_labels_path):
    print("Downloading image labels...")
    download_file('http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat')

In [5]:
image_labels = loadmat(image_labels_path)['labels'][0]
image_labels -= 1

In [6]:
files = sorted(glob.glob(os.path.join(data_path, 'jpg', '*.jpg')))
labels = np.array([i for i in zip(files, image_labels)])

/content/v_d


In [7]:
cwd = os.getcwd()

In [8]:
def move_files(dir_name,cwd,labels):
    cur_dir_path = os.path.join(cwd, dir_name)
    if not os.path.exists(cur_dir_path):
        os.mkdir(cur_dir_path)
    for i in range(0, 102):
        class_dir = os.path.join(cwd, dir_name, str(i))
        os.mkdir(class_dir)
    for label in labels:
        src = str(label[0])
        dst = os.path.join(cwd,dir_name, label[1], src.split(os.sep)[-1])
        copyfile(src, dst)

In [ ]:
dir_name=os.path.join(data_path,'class')
move_files(dir_name,cwd,labels)

In [10]:
def save_dict(content,filename):
    content = dict(content)
    with open(filename,'w') as file_object:
        json.dump(content,file_object)

In [11]:
def load_dict(filename):
    with open(filename,'r') as file_object:
        content = json.load(file_object)
    return content

In [12]:
save_dict(labels,os.path.join(data_path,'image-label.json'))

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Path to the Flowers dataset directory
dataset_directory = '/content/flower/class'

# Load the dataset
dataset = ImageFolder(dataset_directory, transform=transform)

# Define the dataloader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Load pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)
model.to(device)

# Modify the last fully connected layer for the number of flower classes
num_classes = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

print("Training complete!")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 2.9955
Epoch 2/10, Loss: 0.9663
Epoch 3/10, Loss: 0.3772
Epoch 4/10, Loss: 0.1940
Epoch 5/10, Loss: 0.1162
Epoch 6/10, Loss: 0.0756
Epoch 7/10, Loss: 0.0553
Epoch 8/10, Loss: 0.0405
Epoch 9/10, Loss: 0.0326
Epoch 10/10, Loss: 0.0287
Training complete!
